Introduction à Rust

# Séance 5 – Traits

[Pierre-Antoine Champin](http://champin.net/)

Département Info Doua – [IUT Lyon 1](http://iut.univ-lyon1.fr/)


## Bibliothèque standard

### Chemins d'accès

À la séance précédente, on a vu comment utiliser un élément défini dans un module.

In [2]:
mod foo {
    pub fn bar() -> i32 { 42 }
}

foo::bar()

42

In [3]:
use foo::bar;
bar()

42

### Chemin absolu

```rust
// dans le projet (crate) en cours
use crate::foo::bar;

// dans les bibliothèques standard
use std::collections::HashMap;

// dans un crate importé depuis Cargo.toml
use regex::Regex;
```

### Prélude

Certains éléments de la bibliothèque standard sont automatiquement importés,
grâce à un fichier standard nommé `prelude.rs`.

Exemple :

* `Vec` (`std::vec::Vec`)
* `String` (`std::string::String`)
* `Option` (`std::option::Option`)
* `None` et `Some` (`std::option::Option::None`...)
* ...


## Traits 🦀

### Présentation

Les traits sont similaires aux interfaces en Java:

* ils définissent un ensemble de méthodes abstraites,
* ils peuvent être implémentés par plusieurs types indépendants.

In [4]:
pub trait Animal {
    fn couleur(&self) -> &str;
    fn cri(&self) -> &str;
}

NB: les méthodes d'un trait sont toutes considérées comme publiques.

In [5]:
pub struct Corbeau {}

impl Animal for Corbeau {
    fn couleur(&self) -> &str {
        "noir"
    }
    fn cri(&self) -> &str {
        "croa"
    }
}

In [6]:
pub struct Chien { nom: String, couleur: String }

impl Animal for Chien {
    fn couleur(&self) -> &str {
        &self.couleur
    }
    fn cri(&self) -> &str {
        "ouaf"
    }
}

NB: contrairement aux interfaces Java, les traits Rust peuvent contenir autre chose que des fonctions:

* constantes
* types

### Héritage de traits

Certains traits peuvent dépendre d'un ou plusieurs super-traits.

Cela signifie que tout type implémentant ce trait doit également implémenter ses super-traits.

In [7]:
pub trait AnimalDomestique: Animal {
    fn nom(&self) -> &str;
}

NB: cette forme d'héritage ne permet **pas** la surcharge de méthode, uniquement l'ajout de nouvelles méthodes.

In [8]:
impl AnimalDomestique for Chien {
    fn nom(&self) -> &str {
        &self.nom
    }
}

### Implémentation par défaut

Certaines méthodes d'un trait peuvent être munies d'une implémentation par défaut,

qui ne dépend que des méthodes du trait (et de ses super-traits éventuels).


In [9]:
pub trait Rectangulaire {
    fn largeur(&self) -> f64;
    fn longueur(&self) -> f64;
    fn surface(&self) -> f64 {
        self.largeur()*self.longueur()
    }
    fn perimetre(&self) -> f64 {
        2.0*(self.largeur()+self.longueur())
    }
}

### Implémentation sur des types existants 🦀

In [10]:
pub trait Chiffres {
    fn nb_chiffres(&self) -> u32;
}

impl Chiffres for i32 {
    fn nb_chiffres(&self) -> u32 {
        let mut x = *self;
        let mut n = 0;
        while x != 0 {
            n += 1;
            x /= 10;
        }
        n
    }
}

12345.nb_chiffres()

5

## Traits standards

### [`std::clone::Clone`](https://doc.rust-lang.org/std/clone/trait.Clone.html)

Définit une méthode `clone(&self)` permettant de dupliquer une valeur.

NB: certains types n'implémentent pas `Clone`, car une telle duplication n'a pas toujours de sens.

In [11]:
let v1 = vec![1, 2, 3];
let mut v2 = v1.clone();
v2.push(4);
println!("{:?} {:?}", v1, v2);

[1, 2, 3] [1, 2, 3, 4]


### [`std::default::Default`](https://doc.rust-lang.org/std/default/trait.Default.html)
    
Définit une fonction associée `default()`, construisant une valeur de ce type, considérée comme une bonne valeur par défaut.

In [12]:
i32::default()

0

In [13]:
String::default()

""

In [14]:
Option::<i32>::default()

None

In [15]:
Vec::<i32>::default()

[]

### [`std::cmp::PartialEq`](https://doc.rust-lang.org/std/cmp/trait.PartialEq.html)

Définit une méthode `eq(&self, other)` indiquant si `self` et `other` sont égaux. Cette relation *doit* être symétrique et transitive.

**L'opérateur `==` est un raccourci pour l'utilisation de la méthode `PartialEq::eq`.**

Il existe aussi le trait [`PartialOrd`](https://doc.rust-lang.org/std/cmp/trait.PartialOrd.html) pour les opérateurs `<`, `>`, `<=`, `>=`.

### [`std::ops::Add`](https://doc.rust-lang.org/std/ops/trait.Add.html)

Définit une méthode `add(self, other)` retournant la somme de `self` et `other`.

**L'opérateur `+` est un raccourci pour l'utilisation de la méthode `Add::add`.**

Il existe aussi les traits [`Sub`](https://doc.rust-lang.org/std/ops/trait.Sub.html) pour l'opérateur `-`,
[`Mul`](https://doc.rust-lang.org/std/ops/trait.Mul.html) pour `*`,
[`Div`](https://doc.rust-lang.org/std/ops/trait.Div.html) pour `/`.

### [`std::fmt::Debug`](https://doc.rust-lang.org/std/fmt/trait.Debug.html)

Définit une méthode `fmt(&self, ...)` qui définit comment afficher `self` pour le développeur.

**Le format `{:?}` de `prinln!` utilise en interne la méthode `Debug::fmt`.**

Il existe aussi le trait [`Display`](https://doc.rust-lang.org/std/fmt/trait.Display.html) pour le format `{}`.

### [`std::iter::Iterator`](https://doc.rust-lang.org/std/iter/trait.Iterator.html)

Définit une méthode `next(&mut self)` qui retourne `Some(v)` (la prochaine valeur à itérer) ou `None`.

**La boucle `for` utilise en interne la méthode `Iterator::next`.**



NB: `Iterator` fournit également une grande quantité de méthodes munies d'une implémentation par défaut.

In [16]:
let v = vec![2, 4, 1, 3];
v.iter().max()

Some(4)

In [17]:
v.iter().sum::<i32>()

10

### [`std::marker::Copy`](https://doc.rust-lang.org/std/marker/trait.Copy.html)

Un trait *marqueur* ne définit aucune méthode, mais porte une *sémantique* particulière pour le compilateur.

Un type implémentant `Copy` indique au compilateur que

* les valeurs de ce type peuvent être dupliquée sans risque par une simple copie mémoire, et donc
* que le compilateur peut *copier* ces valeurs lors qu'une affectation. 

### Dérivation

Certains traits peuvent être implémentés automatiquement pour un type donné, grâce à la directive `derive`.

In [18]:
#[derive(Clone, Copy, Debug)]
pub struct Point {
    x: f64,
    y: f64,
}

let p1 = Point { x: 1.0, y: 2.0 };
let p2 = p1;
println!("{:?} {:?}", p1, p2);

Point { x: 1.0, y: 2.0 } Point { x: 1.0, y: 2.0 }


⚠ Ceci n'est possible que si les champs du type implémentent eux même les traits en question.

## Traits et généricité

### Fonction polymorphique

In [19]:
/// Retourne l'indice de la plus petite valeur de a
fn imin<T>(a: &[T]) -> usize 
where
    T: PartialOrd,
{
    let mut im = 0;
    for i in 1..a.len() {
        if a[i] < a[im] {
            im = i;
        }
    }
    im
}

La clause `where` contient des *contraintes* sur le(s) trait(s) que doit implémenter le type `T`.

En contrepartie, elles autorisent à utiliser les méthodes de ce trait dans le corps de la fonction
(ici: `a[i] < a[im]` grâce au trait `PartialOrd`).

### Monomorphisation

In [20]:
imin(&[2, -1, 3])

1

In [21]:
imin(&["toto", "titi", "tata"])

2

Le compilateur produit une version de la fonction par type concret avec lequel elle est appelée.

→ par de surcoût à l'exécution lié à l'utilisation des traits

### Fonctions homonymes (*alla* Java)

En Java, plusieurs fonctions/méthodes avec le même nom peuvent co-exister :
```java
void toto(int i) { System.out.println("Vous m'avez donné l'entier " + i); }
void toto(String s) { System.out.println("Vous m'avez donné la chaîne " + s); }
```

En Rust, on peut utiliser les traits pour obtenir le même résultat.

In [22]:
pub trait Description {
    fn description(&self) -> String; 
}

fn toto<D>(d: D)
where
    D: Description
{
    println!("Vous m'avez donné {}", d.description());
}

In [23]:
impl Description for i32 {
    fn description(&self) -> String {
        format!("l'entier {}", self)
    }
}

impl Description for &'_ str {
    fn description(&self) -> String {
        format!("la chaîne {}", self)
    }
}

In [24]:
toto(42);

Vous m'avez donné l'entier 42


Avantage : quelqu'un d'autre peut implémenter le trait `Description` pour son propre type.

In [25]:
impl Description for Point {
    fn description(&self) -> String {
        format!("le point ({}, {})", self.x, self.y)
    }
}

In [26]:
toto(Point{ x: 1.0, y: 2.5 });

Vous m'avez donné le point (1, 2.5)


# Quoi d'autre ?

### Itérateurs

In [27]:
let a = [1, 2, 3];
{
    let v: Vec<_> = a
            .iter()
            .rev()
            .cycle()
            .skip(1)
            .take(6)
            .collect();
    v
}


[2, 1, 3, 2, 1, 3]

### Fermetures

In [28]:
let a = [1, 2, 1, 3, 1, 4, 1, 5];
{
    let v: Vec<_> = a
        .iter()
        .map(|x| 2*x)
        //.take_while(|x| *x<7)
        //.filter(|x| *x>2)
        .collect();
    v
}

[2, 4, 2, 6, 2, 8, 2, 10]

### Threads

In [29]:
use std::thread;
use std::time::Duration;

static values: [i32; 3] = [2, 4, 6];

thread::spawn(|| {
    for v in &values {
        thread::sleep(Duration::from_millis(1000));
        println!("A {}", v);
    }
});

thread::sleep(Duration::from_millis(1500));
for v in &values {
    thread::sleep(Duration::from_millis(1000));
    println!("B {}", v);
}


A 2
A 4
B 2
A 6
B 4
B 6


()